## Data investigation

Data import

Import libs

In [141]:
import sys
import os
import re
import time
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
plt.style.use('ggplot')
import pandas as pd
import numpy as np
import pdb
from datetime import datetime

In [142]:
pwd()

'C:\\Users\\C18244A\\Documents\\Datathon_Global_2018\\notebooks'

In [143]:
## Import the sample data
## clock the time for import

tic = time.time()

df = pd.read_csv("..//data//20180920_Kaufland_case_IoT_and_predictive_maintenance.csv", \
                 sep = ';', encoding='utf-8')

tac = time.time()

diff = tac - tic

print("Time imprt had taken {a:.2f} seconds".format(a = diff))

Time imprt had taken 7.67 seconds


# group by machine, sensor and day and create a column for the max measurement for the particular hour.

In [145]:
## Create a copy of the data
df_copy = df.copy()

In [146]:
## Describe data
df.shape

(3860434, 7)

In [147]:
## Look at the column names
df.columns

Index(['ifm_analysis_sensordata_for_datathon_final.machine_name',
       'ifm_analysis_sensordata_for_datathon_final.sensor_type',
       'ifm_analysis_sensordata_for_datathon_final.date_measurement',
       'ifm_analysis_sensordata_for_datathon_final.start_timestamp',
       'ifm_analysis_sensordata_for_datathon_final.end_timestamp',
       'ifm_analysis_sensordata_for_datathon_final.realvalue',
       'ifm_analysis_sensordata_for_datathon_final.unit'],
      dtype='object')

In [148]:
# Rename columns

In [149]:
df.columns = [x.split('.')[1] for x in df.columns.tolist()]

In [150]:
df.keys()

Index(['machine_name', 'sensor_type', 'date_measurement', 'start_timestamp',
       'end_timestamp', 'realvalue', 'unit'],
      dtype='object')

In [151]:
df.describe()

,realvalue
count,3.860434e+06
mean,1.221669e+03
std,1.893225e+03
min,2.000000e-03
25%,2.379000e+00
50%,7.261400e+02
75%,1.357672e+03
max,2.407599e+04


In [152]:
df.head(3)

,machine_name,sensor_type,date_measurement,start_timestamp,end_timestamp,realvalue,unit
0,RBG1,drive_gear_V_eff,2016-09-02,2016-09-02 15:26:42.823,2016-09-02 15:26:42.823,0.395,mm/s
1,RBG1,drive_gear_V_eff,2016-09-02,2016-09-02 15:26:45.653,2016-09-02 15:26:45.653,0.577,mm/s
2,RBG1,drive_gear_V_eff,2016-09-02,2016-09-02 15:26:48.467,2016-09-02 15:26:48.467,0.717,mm/s


In [153]:
df.sensor_type.unique()

array(['drive_gear_V_eff', 'drive_gear_a_max', 'drive_motor_V_eff',
       'drive_motor_a_max', 'drive_wheel_V_eff', 'drive_wheel_a_max',
       'idle_wheel_V_eff', 'idle_wheel_a_max', 'lifting_gear_V_eff',
       'lifting_gear_a_max', 'lifting_motor_V_eff', 'lifting_motor_a_max'],
      dtype=object)

In [154]:
df.machine_name.unique().tolist()

['RBG1', 'RBG2', 'RBG3', 'RBG4', 'RBG5', 'RBG6', 'RBG7']

In [156]:
df.machine_name.value_counts()

RBG7    1264573
RBG1     928255
RBG2     346060
RBG3     340404
RBG4     332969
RBG6     330710
RBG5     317463
Name: machine_name, dtype: int64

In [157]:
## Save all the groups
# for machine in df.machine_name.unique().tolist():
#     df[df.machine_name == machine].to_csv(f"..//data//{machine}.csv", index=False)

In [158]:
df.dtypes

machine_name         object
sensor_type          object
date_measurement     object
start_timestamp      object
end_timestamp        object
realvalue           float64
unit                 object
dtype: object

In [159]:
df.keys()

Index(['machine_name', 'sensor_type', 'date_measurement', 'start_timestamp',
       'end_timestamp', 'realvalue', 'unit'],
      dtype='object')

In [160]:
## Convrt to datetime
df["date_measurement_f"] = pd.to_datetime(df["date_measurement"])

## Convrt to datetime
df["start_timestamp_f"] = pd.to_datetime(df["start_timestamp"])

## Convrt to datetime
df["end_timestamp_f"] = pd.to_datetime(df["end_timestamp"])

In [161]:
df.dtypes

machine_name                  object
sensor_type                   object
date_measurement              object
start_timestamp               object
end_timestamp                 object
realvalue                    float64
unit                          object
date_measurement_f    datetime64[ns]
start_timestamp_f     datetime64[ns]
end_timestamp_f       datetime64[ns]
dtype: object

In [162]:
df.head(3)

,machine_name,sensor_type,date_measurement,start_timestamp,end_timestamp,realvalue,unit,date_measurement_f,start_timestamp_f,end_timestamp_f
0,RBG1,drive_gear_V_eff,2016-09-02,2016-09-02 15:26:42.823,2016-09-02 15:26:42.823,0.395,mm/s,2016-09-02,2016-09-02 15:26:42.823,2016-09-02 15:26:42.823
1,RBG1,drive_gear_V_eff,2016-09-02,2016-09-02 15:26:45.653,2016-09-02 15:26:45.653,0.577,mm/s,2016-09-02,2016-09-02 15:26:45.653,2016-09-02 15:26:45.653
2,RBG1,drive_gear_V_eff,2016-09-02,2016-09-02 15:26:48.467,2016-09-02 15:26:48.467,0.717,mm/s,2016-09-02,2016-09-02 15:26:48.467,2016-09-02 15:26:48.467


In [163]:
df.drop(['date_measurement', 'start_timestamp', 'end_timestamp'], inplace=True, axis=1)

In [164]:
df.columns

Index(['machine_name', 'sensor_type', 'realvalue', 'unit',
       'date_measurement_f', 'start_timestamp_f', 'end_timestamp_f'],
      dtype='object')

In [165]:
df['measurement_hour'] = df.start_timestamp_f.apply(lambda x: x.hour)

In [166]:
df.head()

,machine_name,sensor_type,realvalue,unit,date_measurement_f,start_timestamp_f,end_timestamp_f,measurement_hour
0,RBG1,drive_gear_V_eff,0.395,mm/s,2016-09-02,2016-09-02 15:26:42.823,2016-09-02 15:26:42.823,15
1,RBG1,drive_gear_V_eff,0.577,mm/s,2016-09-02,2016-09-02 15:26:45.653,2016-09-02 15:26:45.653,15
2,RBG1,drive_gear_V_eff,0.717,mm/s,2016-09-02,2016-09-02 15:26:48.467,2016-09-02 15:26:48.467,15
3,RBG1,drive_gear_V_eff,0.832,mm/s,2016-09-02,2016-09-02 15:26:51.293,2016-09-02 15:26:51.293,15
4,RBG1,drive_gear_V_eff,0.941,mm/s,2016-09-02,2016-09-02 15:26:54.107,2016-09-02 15:26:54.107,15


In [167]:
df.drop(['start_timestamp_f', 'end_timestamp_f'], inplace=True, axis=1)

In [168]:
df.head()

,machine_name,sensor_type,realvalue,unit,date_measurement_f,measurement_hour
0,RBG1,drive_gear_V_eff,0.395,mm/s,2016-09-02,15
1,RBG1,drive_gear_V_eff,0.577,mm/s,2016-09-02,15
2,RBG1,drive_gear_V_eff,0.717,mm/s,2016-09-02,15
3,RBG1,drive_gear_V_eff,0.832,mm/s,2016-09-02,15
4,RBG1,drive_gear_V_eff,0.941,mm/s,2016-09-02,15


In [169]:
import numpy as np

In [170]:
table = pd.pivot_table(df, values=['realvalue'], index=['machine_name','sensor_type', 'date_measurement_f', 'measurement_hour'],
                         aggfunc={'realvalue': np.max})

In [171]:
table.head()

realvalue
machine_name sensor_type      date_measurement_f measurement_hour           
RBG1         drive_gear_V_eff 2016-09-02         15                    1.258
                                                 18                    1.620
                              2016-09-06         10                    1.717
                                                 12                    1.702
                                                 14                    1.880

In [172]:
table.reset_index(inplace=True)

In [173]:
table.head()

,machine_name,sensor_type,date_measurement_f,measurement_hour,realvalue
0,RBG1,drive_gear_V_eff,2016-09-02,15,1.258
1,RBG1,drive_gear_V_eff,2016-09-02,18,1.620
2,RBG1,drive_gear_V_eff,2016-09-06,10,1.717
3,RBG1,drive_gear_V_eff,2016-09-06,12,1.702
4,RBG1,drive_gear_V_eff,2016-09-06,14,1.880


In [174]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51673 entries, 0 to 51672
Data columns (total 5 columns):
machine_name          51673 non-null object
sensor_type           51673 non-null object
date_measurement_f    51673 non-null datetime64[ns]
measurement_hour      51673 non-null int64
realvalue             51673 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 2.0+ MB


In [175]:
table.sensor_type = table.sensor_type.astype(str)

table.machine_name = table.machine_name.astype(str)

table.date_measurement_f = table.date_measurement_f.astype(str)

In [176]:
table['new_index'] = table[['machine_name', 'sensor_type', 'date_measurement_f']].apply(lambda x: '-'.join(x), axis=1)

In [177]:
table1 = table.pivot(index = 'new_index', columns='measurement_hour', values='realvalue')

In [178]:
table2 = table1.reset_index()

In [179]:
table2.columns = ['new_index','0', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']

In [180]:
table2.columns

Index(['new_index', '0', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14',
       '15', '16', '17', '18', '19', '20', '21', '22', '23'],
      dtype='object')

In [181]:
def filter_out(row):
    return [r for r in row if str(r) != 'nan']

In [182]:
 table1['list'] = list(map(filter_out, table2[['0', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']].values.tolist()))

In [183]:
table1['list']

new_index
RBG1-drive_gear_V_eff-2016-09-02                                           [1.258, 1.62]
RBG1-drive_gear_V_eff-2016-09-06         [1.7169999999999999, 1.702, 1.88, 1.956, 1.831]
RBG1-drive_gear_V_eff-2016-09-07        [1.767, 1.743, 1.736, 1.8969999999999998, 1.874]
RBG1-drive_gear_V_eff-2016-09-08               [1.8780000000000001, 1.757, 1.913, 2.013]
RBG1-drive_gear_V_eff-2016-09-09       [2.06, 1.9709999999999999, 1.874, 1.828, 1.83,...
RBG1-drive_gear_V_eff-2016-09-12        [1.771, 1.827, 1.8519999999999999, 1.743, 1.706]
RBG1-drive_gear_V_eff-2016-09-13               [1.777, 2.044, 1.969, 1.8719999999999999]
RBG1-drive_gear_V_eff-2016-09-14       [1.86, 1.9169999999999998, 1.96, 1.80199999999...
RBG1-drive_gear_V_eff-2016-09-15        [1.788, 1.696, 1.771, 1.8459999999999999, 1.844]
RBG1-drive_gear_V_eff-2016-09-16                [1.7169999999999999, 1.8940000000000001]
RBG1-drive_gear_V_eff-2016-09-19                      [1.915, 1.902, 1.9980000000000002]
RBG1-drive_

In [184]:
df.groupby(['machine_name', 'date_measurement_f', 'sensor_type']).nunique()

machine_name  \
machine_name date_measurement_f sensor_type                         
RBG1         2016-09-02         drive_gear_V_eff                1   
                                drive_gear_a_max                1   
                                drive_motor_V_eff               1   
                                drive_motor_a_max               1   
                                drive_wheel_V_eff               1   
                                drive_wheel_a_max               1   
                                idle_wheel_V_eff                1   
                                idle_wheel_a_max                1   
                                lifting_gear_V_eff              1   
                                lifting_gear_a_max              1   
                                lifting_motor_V_eff             1   
                                lifting_motor_a_max             1   
             2016-09-06         drive_gear_V_eff                1   
                                drive_gear_a_max                1   
                                drive_motor_V_eff               1   
                                drive_motor_a_max               1   
                                drive_wheel_V_eff               1   
                                drive_wheel_a_max               1   
                                idle_wheel_V_eff                1   
                                idle_wheel_a_max                1   
                                lifting_gear_V_eff              1   
                                lifting_gear_a_max              1   
                                lifting_motor_V_eff             1   
                                lifting_motor_a_max             1   
             2016-09-07         drive_gear_V_eff                1   
                                drive_gear_a_max                1   
                                drive_motor_V_eff               1   
                                drive_motor_a_max               1   
                                drive_wheel_V_eff               1   
                                drive_wheel_a_max               1   
...                                                           ...   
RBG7         2018-07-08         idle_wheel_V_eff                1   
                                idle_wheel_a_max                1   
                                lifting_gear_V_eff              1   
                                lifting_gear_a_max              1   
                                lifting_motor_V_eff             1   
                                lifting_motor_a_max             1   
             2018-07-09         drive_gear_V_eff                1   
                                drive_gear_a_max                1   
                                drive_motor_V_eff               1   
                                drive_motor_a_max               1   
                                drive_wheel_V_eff               1   
                                drive_wheel_a_max               1   
                                idle_wheel_V_eff                1   
                                idle_wheel_a_max                1   
                                lifting_gear_V_eff              1   
                                lifting_gear_a_max              1   
                                lifting_motor_V_eff             1   
                                lifting_motor_a_max             1   
             2018-07-10         drive_gear_V_eff                1   
                                drive_gear_a_max                1   
                                drive_motor_V_eff               1   
                                drive_motor_a_max               1   
                                drive_wheel_V_eff               1   
                                drive_wheel_a_max               1   
                                idle_wheel_V_eff                1   
                                idle_wheel_a_max   